## Notebook Setup and Uploading of Data to S3

In [12]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
!pip install -Uq altair torchtext torchdata

In [14]:
import sagemaker

In [15]:
!mkdir -p nn_src
import sys
sys.path.append('nn_src')

In [16]:
!touch nn_src/train_dummy
BUCKET = sagemaker.Session().default_bucket()
s3_data_url = f's3://{BUCKET}/nn/'
!aws s3 sync nn_src {s3_data_url} --exclude '*' --include 'train_dummy'

upload: nn_src/train_dummy to s3://sagemaker-eu-west-1-811243659808/nn/train_dummy


In [17]:
%%writefile nn_src/requirements.txt
torchtext
torchdata

Overwriting nn_src/requirements.txt


In [18]:
%%writefile nn_src/trenc.py

import math

import torch
from torch import nn
import torch.nn.functional as F 
from torch import log

class EncoderLayer(nn.Module):
    def __init__(self,
                 d,
                 num_heads,
                 dropout,
                 res_connections=True):

        super().__init__()

        projection_size = d // num_heads

        self.WQ = nn.Parameter(torch.randn(num_heads, d, projection_size)/math.sqrt(d))
        self.WK = nn.Parameter(torch.randn(num_heads, d, projection_size)/math.sqrt(d))
        self.WV = nn.Parameter(torch.randn(num_heads, d, projection_size)/math.sqrt(d))
        self.WO = nn.Parameter(torch.randn(num_heads, projection_size, d)/math.sqrt(d))

        self.attn_norm = nn.LayerNorm(d)
        self.ff_norm   = nn.LayerNorm(d)
        self.dropout   = nn.Dropout(p=dropout)
        self.ff        = nn.Sequential(
            nn.Linear(d, 4*d), 
            nn.GELU(),
            nn.Dropout(p=dropout),
            nn.Linear(4*d, d)
        )

        self.res_connections = res_connections
        # Why just one ReLU/GELU, one dropout? Attention is all you need: 4.3, 5.4. 
        
    def forward(self, input):

        X, prv_attention_values = input

        # b batch
        # t step
        # d dimension
        # p projected dimension
        # h head
        
        normalized_X = self.attn_norm(X) # Pre-Norm

        # Creating keys, queries and values from the normalized values, while
        # at the same time down projecting from d dimensions to p dimensions
        keys    = torch.einsum('btd,hdp->bthp', normalized_X, self.WK)
        queries = torch.einsum('btd,hdp->bthp', normalized_X, self.WQ)
        values  = torch.einsum('btd,hdp->bthp', normalized_X, self.WV)

        # Attention values are per position, not per dimension etc.
        dot = torch.einsum('bkhp,bqhp->bhqk', queries, keys)
        scaled_dot = dot / math.sqrt(values.size()[-1])
        attention_values = torch.softmax(scaled_dot, -1)

        assert torch.allclose(attention_values[0, 0, 0].sum(), torch.tensor(1.))

        new_values = torch.einsum('bkhp,bhqk->bqhp', values, attention_values)
        b, t, h, p = new_values.size()

        # Consolidate the invidual head outputs by concenating them
        # and then up projecting from p to d dimensions.
        seq_summary = torch.einsum('bthp,hpd->btd', new_values, self.WO)
        
        # Alternatively: 
        # Concat outputs of the individual heads
        # Using no learnable parameters
        # seq_summary = new_values.view(b, t, -1)

        assert seq_summary.size()[-1] == h*p == X.size()[-1]

        # Residual Connection and Pre-Norm
        if self.res_connections:
            pre_ff = X + self.ff_norm(seq_summary) 
        else:
            pre_ff = self.ff_norm(seq_summary) 

        prv_attention_values.append(attention_values)

        post_ff = self.ff(self.dropout(pre_ff))
        if self.res_connections:
            return post_ff + pre_ff, prv_attention_values
        else:
            return post_ff, prv_attention_values


class TrEnc(nn.Module):
    def __init__(self, 
                 num_classes, 
                 vocab_size, 
                 layers,
                 d,
                 heads,
                 dropout,
                 use_pos_enc,
                 use_cls_token,
                 **kwargs):
        super().__init__()
        
        res_connections=True
        pos_encoding = 'sin' if use_pos_enc else None
        
        embed_size = d*heads 
        self.pooling  = 'cls' if use_cls_token else 'mean'
        
        self.res_connections = res_connections == True
  
        self.embeddings = nn.Embedding(vocab_size, embed_size, padding_idx=1)

        self.encoder_layers = \
            nn.Sequential(
                *[EncoderLayer(embed_size, heads, dropout, res_connections) for _ in range(layers)],)

        # Empricially it was a toss-up if the additional LN helped or not
        # It is part of the Pre-LN Transformer though
                #nn.LayerNorm(embed_size))
        
        # REVIEW: Just have one matrix for all three weight matrices?
        # REVIEW: Maybe do not project the values, but q and k only
        
        self.dropout     = nn.Dropout(p=dropout)
        #self.clf         = nn.Linear(embed_size, num_classes)
        self.clf = nn.Sequential(
            nn.Dropout(dropout),
            nn.LayerNorm(embed_size),
            nn.Linear(embed_size, embed_size//2, bias=False),
            nn.GELU(),

            nn.Dropout(dropout),
            nn.LayerNorm(embed_size//2),
            nn.Linear(embed_size//2, num_classes)
        )
        
        self.pos_encoder = None

        if 'simple' == pos_encoding:
            self.pos_encoder = SimplePositionalEncoding(dropout)
        elif 'sin' == pos_encoding: 
            self.pos_encoder = PositionalEncoding(embed_size, dropout)

    def extra_repr(self):
        return f'residual_connection={self.res_connections}, pooling={self.pooling}'

    def forward(self, X, output_attentions=False):
        # TODO: Unclear how helpful dropout on embeddings is here. Same p? 
        emb = self.dropout(self.embeddings(X))

        if self.pos_encoder:
            emb = self.pos_encoder(emb) 

        z, attention_values = self.encoder_layers((emb, []))
        
        if self.pooling == 'cls':
            logits = self.clf(self.dropout(z[:, 0]))
        elif self.pooling == 'mean':
            logits = self.clf(self.dropout(z.mean(1)))
        else:
            raise ValueError('Only mean and cls are valid valued for pooling.')

        if output_attentions:
            # return attention as b, l, h, q, k
            return logits, torch.stack(attention_values, 0).transpose(0, 1)
        return logits

class SimplePositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.d_model = d_model

    def forward(self, x):
        enc = torch.arange(-1., 1., step=64, device=x.device)[:, None]
        x = x + self.dropout(enc) 
        return x

# From: https://pytorch.org/tutorials/beginner/transformer_tutorial.html
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


Overwriting nn_src/trenc.py


In [19]:
%%writefile nn_src/model.py

import math

import torch.nn as nn
import torch

class SentimentNN(nn.Module):
    def __init__(
        self, 
        num_classes, 
        vocab_size, 
        layers, 
        d, 
        heads, 
        dropout, 
        use_pos_enc, 
        use_cls_token, 
        *args, 
        **kwargs
    ): 
        
        super().__init__()
        
        self.num_classes   = num_classes
        self.use_cls_token = use_cls_token
        
        # the dimensions of the embeddings must be divisible by the number of heads. 
        # Therefore we specify them this way.
        dh = d*heads 
        layer = nn.TransformerEncoderLayer(
            d_model=dh, 
            nhead=heads, 
            dim_feedforward=dh*4,
            norm_first=True,
            activation='gelu',
            dropout=dropout
        )

        self.trenc = nn.TransformerEncoder(
            layer, 
            num_layers=layers
        )
        
        self.clf = nn.Sequential(
            nn.Dropout(dropout),
            nn.LayerNorm(dh),
            nn.Linear(dh, dh//2, bias=False),
            nn.GELU(),
            
            nn.Dropout(dropout),
            nn.LayerNorm(dh//2),
            nn.Linear(dh//2, num_classes)
        )
        
        self.emb = nn.Embedding(vocab_size, dh, padding_idx=1)
        
        self.pos_enc = PositionalEncoding(dh, dropout=dropout) if use_pos_enc else None
            
    def forward(self, x):
        enc = self.emb(x)
        
        if self.pos_enc:
            enc = self.pos_enc(enc)
        
        h = self.trenc(enc)
        if self.use_cls_token:
            h = h[:, 0, :]
        else:
            h = h.mean(-2)
            
        return self.clf(h)
        
        #enc = self.enc(x)[:, 0, :] # Just the first element
        #enc = self.enc(x).mean(-2) # Just the first element
        
        #enc = self.emb(x).mean(-2)
        #return self.clf(enc)
    
    def extra_repr(self):
        return f'num_classes: {self.num_classes} use_pos_enc: {self.pos_enc is not None} use_cls_token: {self.use_cls_token}'
    
# From: https://pytorch.org/tutorials/beginner/transformer_tutorial.html
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5_000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10_000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


Overwriting nn_src/model.py


In [20]:
%%writefile nn_src/util.py

def count_parameters(m, verbose=True):
    total_count = 0
    learnable_count = 0
    if verbose:
        print('Parameters (name, tunable, count):')
    for n, p in m.named_parameters():
        count = p.data.numel() 
        if verbose:
            print(f' {n:60s} {p.requires_grad:5b} {count:>9d}')
        total_count += count
        if p.requires_grad:
            learnable_count += count
    if verbose:
        print(f'Total parameters: {total_count}, thereof learnable: {learnable_count}')
    return total_count, learnable_count

def str2bool(v):
    if isinstance(v, bool):
        return v
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

Overwriting nn_src/util.py


In [21]:
%%writefile nn_src/train.py

import sys
import argparse
import random
import math
from time import time

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchdata.datapipes as dp

from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

from util import str2bool, count_parameters
from model import SentimentNN
from trenc import TrEnc

def parse_args(arg_values_to_parse):
    parser = argparse.ArgumentParser()

    parser.add_argument('--model', type=str, default='SentimentNN') 
    parser.add_argument('--epochs', type=int, default=1) 
    parser.add_argument('--early-stopping-patience', type=int, default=0)
    parser.add_argument('--batch-size', dest='bs', type=int, default=32) 
    parser.add_argument('--lr', type=float, default=5e-5) 
    
    parser.add_argument('--vocab-size',    type=int, default=10_000) 
    parser.add_argument('--max-input-len', type=int, default=64) 
    
    # model
    parser.add_argument('--dropout', dest='m_dropout',type=float, default=0.2) 
    parser.add_argument('--layers',  dest='m_layers',type=int, default=1) 
    parser.add_argument('--heads',   dest='m_heads', type=int, default=8) 
    parser.add_argument('--dim',     dest='m_d',     type=int, default=64) 
    parser.add_argument('--use-cls-token', dest='m_use_cls_token', type=str2bool, default=True)
    parser.add_argument('--use-pos-enc',   dest='m_use_pos_enc',   type=str2bool, default=False)
    
    parser.add_argument('--dummy', dest='_ignore',type=float, default=0.) 
    
    
    # FIXME: Should I add a separate pooler?
        
    return parser.parse_args(arg_values_to_parse)

def main(*arg_values_to_parse):
    
    args = parse_args([str(la) for la in arg_values_to_parse])
    print('Arguments:', args)
  
    dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    
    ### Data
    train_dp, valid_dp = IMDB(root='imdb_data', split=('train', 'test'))
    
    ### Built vocab and tokenizer
    tokenizer = get_tokenizer('basic_english')

    vocab = build_vocab_from_iterator(
        map(tokenizer, (text for (_, text) in train_dp)), specials=['<UNK>', '<PAD>'], 
        max_tokens=args.vocab_size
    )
    vocab.set_default_index(vocab['<UNK>'])
    
    print(f'Vocabulary built with {len(vocab.get_itos())} tokens.')
    
    ### Data Loader
    def vectorize(y, x):
        y = 0 if y == 'neg' else 1
        return (y, vocab(tokenizer(x)))
    
    def collate(data):
        batch_len = min(args.max_input_len, max([len(x) for (_, x) in data]))
        
        padding = [1] * batch_len # <PAD> has index 1
        y, x = zip(*[(y, (x+padding)[:batch_len]) for (y, x) in data])
 
        return torch.LongTensor(y).to(dev), torch.LongTensor(x).to(dev)
    
    train_dl = DataLoader(
        dataset=[vectorize(y, x) for (y, x) in train_dp], 
        batch_size=args.bs, 
        shuffle=True,
        drop_last=True,
        collate_fn=collate)
    del train_dp
    
    valid_ds = [vectorize(y, x) for (y, x) in valid_dp]
    random.shuffle(valid_ds)
    valid_dl = DataLoader(
        dataset=valid_ds[:7500], 
                batch_size=args.bs*2,
                collate_fn=collate)  
    del valid_dp
    del valid_ds
    
    ### Model
    # Instantiate model and pass all command line args that start with m_
    if args.model == 'SentimentNN':
        model = SentimentNN(
            num_classes=2, # neg, pos 
            vocab_size=len(vocab.get_itos()),
            **{k[2:]: v for k, v in vars(args).items() if k.startswith('m_')}, 
        ).to(dev)
    else:
        model = TrEnc(
            num_classes=2, # neg, pos 
            vocab_size=len(vocab.get_itos()),
            **{k[2:]: v for k, v in vars(args).items() if k.startswith('m_')}, 
        ).to(dev)
        
    print('Model instantiated:', model)

    # prints 'learnable: ddd' to stdout
    count_parameters(model)[1]
    
    ### Train Loop
    optimizer = torch.optim.AdamW(model.parameters(), lr=args.lr)
    criterion = nn.CrossEntropyLoss(reduction='sum')
        
    scaler = torch.cuda.amp.GradScaler()
    
    best_valid_loss = math.inf
    best_epoch = None
    
    #FIXME: documentation.
    best_result_store = None 
    
    for epoch in range(args.epochs):
        model.train()
        train_loss_epoch = 0.
        train_count_epoch = 0
        started = time()
    
        for i, (Yb, Xb) in enumerate(train_dl): 
            
            with torch.cuda.amp.autocast():
                logits = model(Xb)
            
                loss = criterion(logits, Yb)
            
            scaler.scale(loss).backward()
            
            scaler.step(optimizer)
            scaler.update()
            
            train_loss_epoch  += loss.item() 
            train_count_epoch += len(Yb)
            optimizer.zero_grad()

            if i % 50 == 0: 
                print(f'i: {i:4d}: batch_train_loss: {loss/len(Yb):6.4f}')
                
        model.eval()
        with torch.no_grad():
            valid_count_epoch = 0
            matched_epoch = 0
            valid_loss_epoch = 0.
            
            for i, (Yb, Xb) in enumerate(valid_dl):
                with torch.cuda.amp.autocast():
                    logits = model(Xb)
                    valid_loss_epoch += criterion(logits, Yb).item()
                    
                    predictions = logits.argmax(-1)
                    matched_epoch += (predictions == Yb).sum().item()
                    valid_count_epoch += len(Yb)

            acc = matched_epoch/valid_count_epoch
        
        log_message = f'ep: {epoch} train_loss: {train_loss_epoch/train_count_epoch:6.5f} valid_loss: {valid_loss_epoch/valid_count_epoch:6.5f} valid_acc: {acc:5.4f} took: {time()-started:5.3f}s'
        print(log_message)
        
        if valid_loss_epoch < best_valid_loss:
            best_valid_loss = valid_loss_epoch
            best_result_store = log_message
            best_epoch = epoch
            
            print(f'epoch {epoch} was the best epoch so far.')
            
        # early stopping 
        print(f'best epoch {best_epoch}, epoch {epoch}, diff {(epoch-best_epoch)}')
        if best_epoch and epoch >= args.early_stopping_patience-1 and (epoch-best_epoch) >= args.early_stopping_patience:
            print(f'No progress for {args.early_stopping_patience} epochs. Stopping early.')
            break
                
    print('End of training.')
    print('Re-reporting best loss epoch:')
    print(best_result_store)

if __name__ == '__main__':
    print('main', sys.argv)
    main(*sys.argv[1:])    

Overwriting nn_src/train.py


!0!2.0
!1!1.5
!2!1.3
!3!1.3 (last improvement)
!4!1.4
!5!1.5
!6!1.6



In [40]:
#!python nn_src/train.py --epochs 20

In [42]:
from nn_src.train import main as fit
fit('--epochs', 200,
    '--vocab-size', 1_000,
    '--batch-size', 128,
    '--lr', 1e-1, 
    '--layers', 1,
    '--dim', 8,
    '--heads', 2,
    '--max-input-len', 256,
    '--model', 'TrEnc',
    '--early-stopping-patience', 2
)

Arguments: Namespace(model='TrEnc', epochs=200, early_stopping_patience=2, bs=128, lr=0.1, vocab_size=1000, max_input_len=256, m_dropout=0.2, m_layers=1, m_heads=2, m_d=8, m_use_cls_token=True, m_use_pos_enc=False, _ignore=0.0)
Vocabulary built with 1000 tokens.
Model instantiated: TrEnc(
  residual_connection=True, pooling=cls
  (embeddings): Embedding(1000, 16, padding_idx=1)
  (encoder_layers): Sequential(
    (0): EncoderLayer(
      (attn_norm): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
      (ff_norm): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.2, inplace=False)
      (ff): Sequential(
        (0): Linear(in_features=16, out_features=64, bias=True)
        (1): GELU(approximate=none)
        (2): Dropout(p=0.2, inplace=False)
        (3): Linear(in_features=64, out_features=16, bias=True)
      )
    )
  )
  (dropout): Dropout(p=0.2, inplace=False)
  (clf): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): LayerNorm((16,

In [ ]:
xxx

In [22]:
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role

In [23]:
metric_definitions = [
    {'Name': 'Epoch',               'Regex': r'ep:\s+(-?[0-9\.]+)'},
    {'Name': 'Valid:Acc',           'Regex': r'valid_acc:\s+(-?[0-9\.]+)'},
    {'Name': 'Train:Loss',          'Regex': r'train_loss:\s+(-?[0-9\.]+)'},
    {'Name': 'Valid:Loss',          'Regex': r'valid_loss:\s+(-?[0-9\.]+)'}, 
    {'Name': 'LearnableParameters', 'Regex': r'learnable:\s+(-?[0-9\.]+)'} 
]                                 

In [24]:
estimator = PyTorch('train.py',
                    source_dir='nn_src',
                    role=get_execution_role(),
                    instance_type= 'ml.g4dn.xlarge',
                    instance_count=1,
                    framework_version='1.10',
                    py_version='py38',
                    metric_definitions=metric_definitions,
                    base_job_name='nn',
                    
                    use_spot_instances= True,
                    max_run=  60 * 60 * 24,
                    max_wait= 60 * 60 * 24,
                    
                    keep_alive_period_in_seconds= 120,
                    
                    hyperparameters = {'early-stopping-patience': 4, 
                                       'epochs': 20,
                                       'batch-size': 128,
                                       'lr': 2e-4,
                                       'vocab-size': 10_000,
                                       'heads': 12,
                                       'dim': 32, 
                                       'max-input-len': 256,
                                       'layers': 1,
                                       'dropout': 0.2, 
                                       'use-cls-token': True,
                                       'use-pos-enc': True,
                                       'model': 'TrEnc'}) # SentimentNN, TrEnc

In [112]:
#estimator.fit({'train': s3_data_url+'train_dummy'}, wait=False)

### Automatic Model Tuning Jobs
#### Random Sweep to check the lay of the land

In [55]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter
from sagemaker.tuner import ContinuousParameter, HyperparameterTuner

hpt_ranges = {
    #'dummy': ContinuousParameter(1e-7, 1e-1), 
    #'vocab-size': IntegerParameter(1_000, 10_000),
    'lr': ContinuousParameter(1e-6, 1e-1), 
    #'max-input-len': IntegerParameter(100, 1_000),
    #'dropout': ContinuousParameter(0.00, 0.6),
    #'heads': IntegerParameter(4, 16),
    #'dim': IntegerParameter(32, 96),
    #'layers': IntegerParameter(1, 12),
    #'model': CategoricalParameter(['SentimentNN', 'TrEnc']),
    #'use-pos-enc': CategoricalParameter([True, False]),
    #'use-cls-token': CategoricalParameter([True, False])
}

tuner_parameters = {'estimator': estimator,
                    'base_tuning_job_name': 'model-lr-nn',
                    'metric_definitions': metric_definitions,
                    
                    'objective_metric_name': 'Valid:Acc',
                    'objective_type': 'Maximize',
                    'hyperparameter_ranges': hpt_ranges,
                    'strategy': 'Bayesian',
                    'early_stopping_type': 'Auto',
                    
                    'max_jobs': 15,           # Was: 20 
                    'max_parallel_jobs': 1}   # Was: 20

In [56]:
tuner = HyperparameterTuner(**tuner_parameters)
tuner.fit({'train': s3_data_url+'train_dummy'}, wait=False)
tuner_name = tuner.describe()["HyperParameterTuningJobName"]
print(f'tuning job submitted: {tuner_name}.')

tuning job submitted: model-lr-nn-221021-1022.


In [25]:
model  = 'model-221020-1931'
layers = 'layers-221020-1931'
md_lr = ['model-lr-tr-221020-2000', 'model-lr-nn-221020-2001', 'model-lr-nn-221021-1022']
pos_enc = 'pos-enc-221020-2003'
pos_enc_lr = ['pe-t-lr-221020-2140', 'pe-f-lr-221020-2141']

In [30]:
!pwd

/Users/mkamp/code/amazon-sagemaker-amt-visualize


In [ ]:
%%time
from reporting_util import analyze_hpo_job
chart, trials_df, full_df = analyze_hpo_job(
    md_lr, 
    return_dfs=True,
    job_metrics=[
        'Train:Loss',
        'Valid:Loss',
        'Epoch',
        'LearnableParameters'
    ]
)
chart 

Tuning job model-lr-tr-221020-2000   status: Completed
Tuning job model-lr-nn-221020-2001   status: Completed
Tuning job model-lr-nn-221021-1022   status: Stopped

Number of training jobs with valid objective: 50
Lowest: 0.5076000094413757 Highest 0.8151000142097473


,lr,TrainingJobName,TrainingJobStatus,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds,TuningJobName,Valid:Acc
10,0.000447,model-lr-tr-221020-2000-010-21c4f7b4,Completed,2022-10-20 22:14:04+02:00,2022-10-20 22:25:54+02:00,710.0,model-lr-tr-221020-2000,0.8151
0,0.000698,model-lr-tr-221020-2000-020-80d7c4e0,Completed,2022-10-21 00:39:21+02:00,2022-10-21 00:51:35+02:00,734.0,model-lr-tr-221020-2000,0.8104
1,0.000478,model-lr-tr-221020-2000-019-599b0f9d,Completed,2022-10-21 00:25:12+02:00,2022-10-21 00:37:12+02:00,720.0,model-lr-tr-221020-2000,0.8079
13,0.000529,model-lr-tr-221020-2000-007-6fb682d4,Completed,2022-10-20 21:31:18+02:00,2022-10-20 21:43:41+02:00,743.0,model-lr-tr-221020-2000,0.8064
11,0.000602,model-lr-tr-221020-2000-009-953d7280,Completed,2022-10-20 21:59:48+02:00,2022-10-20 22:11:58+02:00,730.0,model-lr-tr-221020-2000,0.7997
14,0.000256,model-lr-tr-221020-2000-006-136ed090,Completed,2022-10-20 21:16:50+02:00,2022-10-20 21:29:25+02:00,755.0,model-lr-tr-221020-2000,0.7969
5,0.000801,model-lr-tr-221020-2000-015-cfbc67df,Completed,2022-10-20 23:28:09+02:00,2022-10-20 23:40:19+02:00,730.0,model-lr-tr-221020-2000,0.7968
7,0.000126,model-lr-tr-221020-2000-013-175d72a6,Completed,2022-10-20 22:59:37+02:00,2022-10-20 23:11:46+02:00,729.0,model-lr-tr-221020-2000,0.7955
18,0.000268,model-lr-tr-221020-2000-002-ba9388a9,Completed,2022-10-20 20:16:40+02:00,2022-10-20 20:29:00+02:00,740.0,model-lr-tr-221020-2000,0.7927
8,0.000026,model-lr-tr-221020-2000-012-5ae01165,Completed,2022-10-20 22:45:18+02:00,2022-10-20 22:57:18+02:00,720.0,model-lr-tr-221020-2000,0.7905


In [59]:
XXX

NameError: name 'XXX' is not defined